In [1]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler

In [3]:
titanic = sns.load_dataset('titanic')

In [5]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [7]:
titanic.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [9]:
# count: 해당 열의 값이 존재하는 개수를 나타냅니다. 결측치가 없는 수치를 보여줍니다.
# std: 표준 편차를 나타내며, 데이터의 분산 정도를 나타냅니다. 값이 클수록 데이터가 평균에서 멀리 퍼져 있습니다.
# min: 최솟값을 나타냅니다.
# 25%: 데이터의 하위 25%에 해당하는 값.
# 50%: 데이터의 중앙값을 냄.
# 75%: 데이터의 상위 25%에 해당하는 값.
# max: 최댓값을 나타냄.

In [11]:
print(titanic.isnull().sum())

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64


In [17]:
titanic['age'] = titanic['age'].fillna(titanic['age'].median())
titanic['embarked'] = titanic['embarked'].fillna(titanic['embarked'].mode()[0])

In [19]:
print(titanic['age'].isnull().sum())
print(titanic['embarked'].isnull().sum())

0
0


In [21]:
titanic['sex'] = titanic['sex'].map({'male': 0, 'female': 1})
titanic['alive'] = titanic['alive'].map({'yes': 1, 'no': 0})
titanic['embarked'] = titanic['embarked'].map({'C': 0, 'Q': 1, 'S': 2,})

print(titanic['sex'].head())
print(titanic['alive'].head())
print(titanic['embarked'].head())

0    0
1    1
2    1
3    1
4    0
Name: sex, dtype: int64
0    0
1    1
2    1
3    1
4    0
Name: alive, dtype: int64
0    2
1    0
2    2
3    2
4    2
Name: embarked, dtype: int64


In [23]:
titanic['family_size'] = titanic['sibsp'] + titanic['parch'] + 1

In [25]:
print(titanic['family_size'].head())

0    2
1    2
2    1
3    2
4    1
Name: family_size, dtype: int64


In [27]:
titanic = titanic[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'family_size']]
X = titanic.drop('survived', axis=1) # feature
y = titanic['survived'] # target 분리

In [31]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X) #데이터 스케일링

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42) #random forest 모델 학습

In [41]:
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [37]:
y_pred = rf_model.predict(X_test) #예측

In [43]:
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred)}")

Accuracy: 0.8212290502793296
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.85      0.85       105
           1       0.78      0.78      0.78        74

    accuracy                           0.82       179
   macro avg       0.82      0.82      0.82       179
weighted avg       0.82      0.82      0.82       179



In [45]:
# 모델이 생존하지 않는다고 예측한 것 중 실제로 생존하지 않은 비율이 85%입니다.
# 모델이 생존한다고 예측한 것 중 실제로 생존한 비율이 78%입니다.
# 실제로 생존하지 않은 승객의 수는 105
# 실제로 생존한 승객의 수 74
# 이 모델은 생존 여부를 예측하는 데 있어 좋은 성능을 보이고 있으며, 
# 생존하지 않는 승객을 비교적 잘 예측하고 있습니다. 그러나 생존한 승객(1 클래스)에 대해서는 성능 개선이 필요할 수 있습니다. 
# 예를 들어, 데이터를 추가하거나 모델의 하이퍼파라미터를 조정하여 성능을 향상시킬 수 있습니다.